In [29]:
import yfinance as yf
import pandas as pd

In [ ]:
"""
# Define the ticker symbol
ticker = "^IXIC"  # NASDAQ Composite

# Download historical data
data = yf.download(ticker, start="1990-01-01", end="2024-11-30", interval="1d")

# Save to a CSV file
data.to_csv("19900101_20241130.csv")
print("Data saved to 19900101_20241130.csv")

"""

[*********************100%***********************]  1 of 1 completed

Data saved to 19900101_20241130.csv


In [34]:
df = pd.read_csv('19900101_20241130.csv')

# Step 1: Remove the Ticker row (assumes it is on the first row, index 0)
df = df.iloc[1:]  # Keep all rows starting from index 1 (skips row 0)

# Step 2: Rename the columns to be more concise
df.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']

# Step 3: Remove the first row 
df = df.iloc[1:]

# Step 4: Set the 'Date' column as the index
df = df.set_index('Date')

In [37]:
# Convert every columns into a float type
object_cols = df.select_dtypes(include='object').columns
df[object_cols] = df[object_cols].astype('float')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8796 entries, 1990-01-02 to 2024-11-29
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       8796 non-null   float64
 1   High       8796 non-null   float64
 2   Low        8796 non-null   float64
 3   Close      8796 non-null   float64
 4   Adj Close  8796 non-null   float64
 5   Volume     8796 non-null   float64
dtypes: float64(6)
memory usage: 481.0+ KB


In [38]:
df.head(3)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1990-01-02,459.299988,459.299988,459.299988,452.700012,452.899994,110720000.0
1990-01-03,460.899994,460.899994,461.600006,460.000000,461.100006,152660000.0
1990-01-04,459.399994,459.399994,460.799988,456.899994,460.399994,147950000.0


In [5]:
# Optionally save the cleaned DataFrame back to a CSV
#df.to_csv('cleaned_nasdaq.csv')

##### Functions to engineer new features

In [39]:
def add_original_feature(df):
    df['close_1'] = df['Close'].shift(1)
    df['open_1'] = df['Open'].shift(1)
    df['high_1'] = df['High'].shift(1)
    df['low_1'] = df['Low'].shift(1)
    df['volume_1'] = df['Volume'].shift(1)
    return df

def add_avg_closing_price(df):
    df['avg_close_week'] = df['Close'].rolling(5).mean().shift(1)
    df['avg_close_month'] = df['Close'].rolling(21).mean().shift(1)
    df['avg_close_year'] = df['Close'].rolling(252).mean().shift(1)

    df['ratio_avg_price_week_month'] = df['avg_close_week'] / df['avg_close_month']
    df['ratio_avg_price_week_year'] = df['avg_close_week'] / df['avg_close_year']
    df['ratio_avg_price_month_year'] = df['avg_close_month'] / df['avg_close_year']
    return df

def add_avg_volume(df):
    df['avg_volume_week'] = df['Volume'].rolling(5).mean().shift(1)
    df['avg_volume_month'] = df['Volume'].rolling(21).mean().shift(1)
    df['avg_volume_year'] = df['Volume'].rolling(252).mean().shift(1)

    df['ratio_avg_volume_week_month'] = df['avg_volume_week'] / df['avg_volume_month']
    df['ratio_avg_volume_week_year'] = df['avg_volume_week'] / df['avg_volume_year']
    df['ratio_avg_volume_month_year'] = df['avg_volume_month'] / df['avg_volume_year']
    return df

def add_std_price(df):
    df['std_price_week'] = df['Close'].rolling(5).std().shift(1)
    df['std_price_month'] = df['Close'].rolling(21).std().shift(1)
    df['std_price_year'] = df['Close'].rolling(252).std().shift(1)
    df['ratio_std_price_week_month'] = df['std_price_week'] / df['std_price_month']
    df['ratio_std_price_week_year'] = df['std_price_week'] / df['std_price_year']
    df['ratio_std_price_month_year'] = df['std_price_month'] / df['std_price_year']
    return df

def add_std_volume(df):
    df['std_volume_week'] = df['Volume'].rolling(5).std().shift(1)
    df['std_volume_month'] = df['Volume'].rolling(21).std().shift(1)
    df['std_volume_year'] = df['Volume'].rolling(252).std().shift(1)
    df['ratio_std_volume_week_month'] = df['std_volume_week'] / df['std_volume_month']
    df['ratio_std_volume_week_year'] = df['std_volume_week'] / df['std_volume_year']
    df['ratio_std_volume_month_year'] = df['std_volume_month'] / df['std_volume_year']
    return df

def add_return_feature(df):
    df['daily_return'] = ((df['Close'] - df['Close'].shift(1)) / df['Close'].shift(1)).shift(1)
    df['weekly_return'] = ((df['Close'] - df['Close'].shift(5)) / df['Close'].shift(5)).shift(1)
    df['monthly_return'] = ((df['Close'] - df['Close'].shift(21)) / df['Close'].shift(21)).shift(1)
    df['yearly_return'] = ((df['Close'] - df['Close'].shift(252)) / df['Close'].shift(252)).shift(1)

    df['moving_avg_week'] = df['daily_return'].rolling(5).mean().shift(1)
    df['moving_avg_month'] = df['daily_return'].rolling(21).mean().shift(1)
    df['moving_avg_year'] = df['daily_return'].rolling(252).mean().shift(1)
    return df

def generate_features(df):
    """
    Generate features for a stock/index based on historical price and 
    performance
    @param df: dataframe with columns "Open", "Close", "High", "Low", 
    "Volume", "Adj Close"
    @return: dataframe, data set with new features
    """
    df = add_original_feature(df)
    df = add_avg_closing_price(df)
    df = add_avg_volume(df)
    df = add_std_price(df)
    df = add_std_volume(df)
    df = add_return_feature(df)
    return df

In [40]:
data = generate_features(df)
data.head()

,Open,High,Low,Close,Adj Close,Volume,close_1,open_1,high_1,low_1,...,ratio_std_volume_week_month,ratio_std_volume_week_year,ratio_std_volume_month_year,daily_return,weekly_return,monthly_return,yearly_return,moving_avg_week,moving_avg_month,moving_avg_year
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,459.299988,459.299988,459.299988,452.700012,452.899994,110720000.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,460.899994,460.899994,461.600006,460.000000,461.100006,152660000.0,452.700012,459.299988,459.299988,459.299988,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,459.399994,459.399994,460.799988,456.899994,460.399994,147950000.0,460.000000,460.899994,460.899994,461.600006,...,NaN,NaN,NaN,0.016125,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,458.200012,458.200012,459.399994,457.799988,457.899994,137230000.0,456.899994,459.399994,459.399994,460.799988,...,NaN,NaN,NaN,-0.006739,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,458.700012,458.700012,458.700012,456.500000,457.100006,115500000.0,457.799988,458.200012,458.200012,459.399994,...,NaN,NaN,NaN,0.001970,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
print(df[['daily_return', 'monthly_return', 'yearly_return']])

            daily_return  monthly_return  yearly_return
Date                                                   
1990-01-02           NaN             NaN            NaN
1990-01-03           NaN             NaN            NaN
1990-01-04      0.016125             NaN            NaN
1990-01-05     -0.006739             NaN            NaN
1990-01-08      0.001970             NaN            NaN
...                  ...             ...            ...
2024-11-22     -0.000552        0.031270       0.322893
2024-11-25      0.009908        0.032453       0.328458
2024-11-26      0.003697        0.026088       0.334542
2024-11-27      0.006926        0.028967       0.344626
2024-11-29     -0.008562        0.023102       0.334008

[8796 rows x 3 columns]


In [46]:
df.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'close_1',
       'open_1', 'high_1', 'low_1', 'volume_1', 'avg_close_week',
       'avg_close_month', 'avg_close_year', 'ratio_avg_price_week_month',
       'ratio_avg_price_week_year', 'ratio_avg_price_month_year',
       'avg_volume_week', 'avg_volume_month', 'avg_volume_year',
       'ratio_avg_volume_week_month', 'ratio_avg_volume_week_year',
       'ratio_avg_volume_month_year', 'std_price_week', 'std_price_month',
       'std_price_year', 'ratio_std_price_week_month',
       'ratio_std_price_week_year', 'ratio_std_price_month_year',
       'std_volume_week', 'std_volume_month', 'std_volume_year',
       'ratio_std_volume_week_month', 'ratio_std_volume_week_year',
       'ratio_std_volume_month_year', 'daily_return', 'weekly_return',
       'monthly_return', 'yearly_return', 'moving_avg_week',
       'moving_avg_month', 'moving_avg_year'],
      dtype='object')